In [1]:
import pandas as pd
import numpy as np
import pyodbc as pc
import warnings

# from t2 import *

warnings.filterwarnings('ignore')
path_start = 'c:/tt/'

In [2]:
def marsh_last(dfm, bm):
    query = f"bm=='{bm}' and (em2=='За' or em2=='На') and tm==1"
    return dfm.query(query)['id'].tolist()

def marsh_middle(dfm, bm):
    query = f"bm=='{bm}' and em2=='До' and tm==1"
    return dfm.query(query)['id'].tolist()

def marsh_all_bm(dfm, bm, ls_id):
    ls = []

    ls_k = marsh_last(dfm, bm)
    for id_em in ls_k:
        ls.append(ls_id + [id_em])


    ls_p = marsh_middle(dfm, bm)
    if len(ls_p) == 0:
        return ls
    for pm in ls_p:
        em = dfm[dfm['id']==pm]['em'].iloc[0]
        ls_p_k = marsh_all_bm(dfm, em, ls_id + [pm])
        if ls_p_k:
            ls.extend(ls_p_k)
    return ls

def marsh_all_bm_exclude(ls_marsh_all_bm):
    cnxn = connect()
    dfsv = pd.read_sql('''select * from sv''', cnxn)
    dfrs = pd.read_sql('''select * from rs''', cnxn)
    dfmarsh = pd.read_sql('''select * from devmarsh''', cnxn)
    cnxn.close()

    ls_marsh = []
    for lsm in ls_marsh_all_bm:
        ls_marsh1 = []
        for m_id in lsm:
            dfm1 = df_col(dfmarsh, 'marshrut_id', m_id)
            dfm2 = dfm1[(dfm1['telm']=='Светофор') | (dfm1['telm']=='РЦ')].iloc[1:]
            for index, row in dfm2.iterrows():
                fl = 0
                if row['telm'] == 'Светофор':
                    sr1  = df_name(dfsv, row['elm']).iloc[0]
                    if sr1['pok'] != 0 or sr1['remont'] != 0 or sr1['marsh'] != 0:
                        fl = 1
                if row['telm'] == 'РЦ':
                    sr1  = df_name(dfrs, row['elm']).iloc[0]
                    if sr1['state'] != 0 or sr1['remont'] != 0 or sr1['marsh'] != 0:
                        fl = 1
            if fl == 0:
                ls_marsh1.append(m_id)
            else:
                break
        if len(ls_marsh1) > 0:
            ls_marsh.append(ls_marsh1)
    return ls_marsh

def to_txt_ls(namefile, ls):
    path = path_start + namefile
    with open(path, "w", encoding='Windows-1251') as f:
        for ss in ls:
            f.write(ss +'\n')

def to_txt_ls2(namefile, ls):
    path = path_start + namefile
    with open(path, "w", encoding='Windows-1251') as f:
        for ss in ls:
            s1 = ''
            for ss1 in ss:
                s1 += str(ss1) + ','
            if len(s1) > 0:
                s1 = s1[:-1]
                f.write(s1)
                f.write('\n')

def read_txt_ls(namefile):
    path = path_start + namefile
    ls = []
    with open(path, "r", encoding='Windows-1251') as f:
        for line in f:
            s1 = line.strip()
            ls.append(s1)
    return ls

def read_txt_ls2(namefile):
    path = path_start + namefile
    ls = []
    with open(path, "r", encoding='Windows-1251') as f:
        for line in f:
            ls1 = line.strip().split(',')
            ls1 = [int(x) for x in ls1]
            ls.append(ls1)
    return ls

def marsh_all_bm_in_elem(dfm, bm):
    ls_all_marsh = marsh_all_bm(dfm, bm, [])
    ls_all_marsh = marsh_all_bm_exclude(ls_all_marsh)
    marsh_elem(dfm, ls_all_marsh)

def marsh_all_em_in_elem(dfm, em):
    ls_marsh_id = read_txt_ls2("marsh_all_bm.txt")
    ls_marsh_id = marsh_all_em(dfm, em, ls_marsh_id)
    marsh_elem(dfm, ls_marsh_id)

def marsh_all_pm_in_elem(dfm, pm, em):
    ls_marsh_id = read_txt_ls2("marsh_all_bm.txt")
    ls_marsh_id = marsh_all_pm(dfm, pm, em, ls_marsh_id)
    marsh_elem(dfm, ls_marsh_id)

def marsh_elem(dfm, ls_all_marsh, is_txt=True):
    ls_sv = []
    ls_str = []
    ls_rs = []
    for lm in ls_all_marsh:
        for id_m in lm:
            df1 = dfm[dfm['id']==id_m]
            # SV ------------------------
            bm1 = df1['bm'].iloc[0]
            if bm1 not in ls_sv:
                ls_sv.append(bm1)
            # STR ------------------------
            st1 = df1['sm1'].iloc[0]
            if st1 is not np.nan:
                ls_st = st1.split(',')
                for st2 in ls_st:
                    # st3 = st2[1:]
                    if st2 not in ls_str:
                        ls_str.append(st2)
            # RS ------------------------
            rs1 = df1['rs'].iloc[0]
            if rs1 is not np.nan:
                ls_st = rs1.split(',')
                for rs in ls_st:
                    if rs not in ls_rs:
                        ls_rs.append(rs)

        id_m = lm[-1]
        em = dfm[dfm['id']==id_m]['em'].iloc[0]
        if em not in ls_sv:
            ls_sv.append(em)
    if is_txt:
        to_txt_ls("marsh_sv_bm.txt", ls_sv)
        to_txt_ls("marsh_str_bm.txt", ls_str)
        to_txt_ls("marsh_rs_bm.txt", ls_rs)
        to_txt_ls2("marsh_all_bm.txt", ls_all_marsh)
        to_txt_ls("marsh_is_sel.txt", [])
    else:
        return ls_all_marsh, ls_sv, ls_str, ls_rs

def marsh_all_em(dfm, em, ls_marsh_id):
    ls_marsh_id1 = []
    for ls1 in ls_marsh_id:
        ls2 = []
        for mars_id in ls1:
            ls2.append(mars_id)
            em1 = df_id(dfm, mars_id)['em'].iloc[0]
            if em == em1:
                if ls2 not in ls_marsh_id1:
                    ls_marsh_id1.append(ls2)
                break
    ls_marsh_id1 = sorted(ls_marsh_id1, key=lambda x: len(x))
    return ls_marsh_id1

def marsh_all_pm(dfm, pm, em, ls_marsh_id):
    ls_marsh_id1 = []
    if pm == em:
        ls_marsh_id1.append(ls_marsh_id[0])
    else:
        for ls1 in ls_marsh_id:
            for mars_id in ls1:
                em1 = df_id(dfm, mars_id)['em'].iloc[0]
                if pm == em1:
                    ls_marsh_id1.append(ls1)
                    break
    return ls_marsh_id1

def create_marsh():
    df = pd.read_excel('c:/tt/marsh_fabr.xlsx')
    df['id'] = list(range(1, len(df)+1))
    df = df[['id', 'bm', 'em', 'em2', 'sm', 'sm1', 'rs']]
    df['sm'] = df['sm'].str.replace(' ', '')
    df['sm1'] = df['sm1'].str.replace(' ', '')
    df['rs'] = df['rs'].str.replace(' ', '')
    df['sm'].fillna('', inplace=True)
    df['sm1'].fillna('', inplace=True)
    df['tm'] = 1
    # df[['id', 'bm', 'em', 'em2', 'tm']].to_csv('c:/tt/marsh_fabr.csv', index_label=False)
    df.to_csv('c:/tt/marsh_fabr.csv', index_label=False)
    return df

def connect():
    connection_string = "Driver=SQL Server;Server=localhost;Database=fabr;Trusted_Connection=Yes;"
    cnxn = pc.connect(connection_string, autocommit=True)
    return cnxn

def df_id(df, id1):
    return df[df['id']==id1]

def df_name(df, name):
    return df[df['name']==name]

def df_col(df, col, val):
    return df[df[col]==val]

def read_marsh():
    df = pd.read_csv('c:/tt/marsh_fabr.csv')
    df['sm'] = df['sm'].str.replace(' ', '')
    df['sm1'] = df['sm1'].str.replace(' ', '')
    df['rs'] = df['rs'].str.replace(' ', '')
    df['sm'].fillna('', inplace=True)
    df['sm1'].fillna('', inplace=True)
    df['smls'] = df['sm'].apply(lambda x: x.split(','))
    df['sm1ls'] = df['sm1'].apply(lambda x: x.split(','))
    df['rsls'] = df['rs'].apply(lambda x: x.split(','))
    return df

def sv_blok_elmMarsh(idMarsh):
    ls_rs = df[df['id'] == idMarsh]['rsls'].iloc[0][1:]
    lsm = []
    for rs in ls_rs:
        df1 = df.loc[df.apply(lambda x:  rs in x.rsls, axis=1)]
        ls_bm = list(set(df1['bm'].tolist()))
        for bm in ls_bm:
            ls1 = df[df['bm']==bm]['rsls'].tolist()
            fl = 0
            if len(ls1) > 1:
                for ls2 in ls1:
                    if rs not in ls2:
                        fl = 1
            if fl == 0:
                lsm.append(bm)
    lsm = list(set(lsm))
    return lsm

def marsh_sv_blok():
    ls = read_txt_ls2("marsh_all_bm.txt")
    if len(ls) > 0:
        ls = ls[0]
        ls2 = []
        for idMarsh in ls:
            ls2.extend(sv_blok_elmMarsh(idMarsh))
        ls2 = list(set(ls2))
        to_txt_ls("marsh_sv_blok.txt", ls2)

In [113]:
# create_marsh()

In [25]:
cnxn = connect()
dfsv = pd.read_sql('''select * from sv''', cnxn)
dfstr = pd.read_sql('''select * from str''', cnxn)
dfrs = pd.read_sql('''select * from rs''', cnxn)
dfmarsh = pd.read_sql('''select * from devmarsh''', cnxn)
cnxn.close()

In [3]:
df = read_marsh()
df

,id,bm,em,em2,sm,sm1,rs,tm,smls,sm1ls,rsls
0,1,Мн,М1,До,,,"Т,1ГП",1,[],[],"[Т, 1ГП]"
1,2,Мз,М7,До,,,"Т,2ГП",1,[],[],"[Т, 2ГП]"
2,3,М1,М203,До,"-1/3,-5","-1,-3,-5","1ГП,1,3-5,5/201П",1,"[-1/3, -5]","[-1, -3, -5]","[1ГП, 1, 3-5, 5/201П]"
3,4,М1,М11,До,"-1/3,+5","-1,-3,+5","1ГП,1,3-5,5/9П",1,"[-1/3, +5]","[-1, -3, +5]","[1ГП, 1, 3-5, 5/9П]"
4,5,М1,М13,До,+1/3,+1,"1ГП,1,1/15П",1,[+1/3],[+1],"[1ГП, 1, 1/15П]"
...,...,...,...,...,...,...,...,...,...,...,...
289,290,М216,М218,За,"-208,+204/206,-212","-208,+204,-212","203П,204-212",1,"[-208, +204/206, -212]","[-208, +204, -212]","[203П, 204-212]"
290,291,М216,М206,До,"-208,+204/206,+212","-208,+204,+212","203П,204-212,202/212П",1,"[-208, +204/206, +212]","[-208, +204, +212]","[203П, 204-212, 202/212П]"
291,292,М218,М212,На,"-212,-204/206","-212,-204,-206","Т,204-212,206,201П",1,"[-212, -204/206]","[-212, -204, -206]","[Т, 204-212, 206, 201П]"
292,293,М218,М214,На,"-212,+204/206,+208","-212,+204,+208","Т,204-212,202П",1,"[-212, +204/206, +208]","[-212, +204, +208]","[Т, 204-212, 202П]"


In [112]:
marsh_sv_blok()

In [114]:
read_txt_ls("marsh_sv_blok.txt")

['М9',
 'М43',
 'М63',
 'М5',
 'М41',
 'М7',
 'М11',
 'М23',
 'М35',
 'М15',
 'М17',
 'М19',
 'М201']

In [29]:
em = 'М11'

In [97]:
bm = 'М11'
# ls_marsh_id = marsh_all_bm(df, bm, [])
# to_txt_ls2("marsh_all_bm.txt", ls_marsh_id)
# ls_marsh_id, *pp = marsh_all_bm_in_elem(df, bm, is_txt=False)
marsh_all_bm_in_elem(df, bm)

In [98]:
em = 'М63'
ls_marsh_id1 = read_txt_ls2("marsh_all_bm.txt")
ls_marsh_id1 = marsh_all_em(df, em, ls_marsh_id1)
ls_marsh_id1

[[15, 29], [15, 28, 34], [16, 30, 34]]

In [107]:
marsh_elem(df, ls_marsh_id1)

In [17]:
bm = 'М11'
marsh_all_bm_in_elem(df, bm)

In [18]:
em = 'М63'
marsh_all_em_in_elem(df, em)

In [19]:
pm = 'М23'
marsh_all_pm_in_elem(df, pm, em)

In [20]:
pm = em
marsh_all_pm_in_elem(df, pm, em)

In [12]:
df_id(df, 30)

,id,bm,em,em2,sm,sm1,rs,tm
29,30,М25,М29,До,+29/31,+31,"25/31П,31",1


In [40]:
dfm1 = df_col(dfmarsh, 'marshrut_id', 33)
dfm1

,id,marshrut_id,elm,telm,ppos,isp,pnisp
231,232,33,31,РЦ,1,0,1
232,233,33,М29,Светофор,1,0,2
233,234,33,33-43,РЦ,0,1,3
234,235,33,33/35,Стрелка,1,0,0
235,236,33,41,Стрелка,0,0,0
236,237,33,43,Стрелка,1,0,0


In [24]:
dfm1[(dfm1['telm']=='Светофор') | (dfm1['telm']=='РЦ')].iloc[1:]

,id,marshrut_id,elm,telm,ppos,isp,pnisp
61,62,12,М11,Светофор,1,0,2
62,63,12,9,РЦ,0,1,3


In [29]:
d1 = df_name(dfsv, 'М11')
d1

,id,name,pok,type,fault_B,fault_K,remont,marsh
7,8,М11,0,0,0,0,0,0


In [48]:
dfrs.loc[dfrs['name']=='33-43', 'state'] = 0

In [104]:
bm = 'М1'
ls_sv, ls_str, ls_rs = marsh_all_bm_in_elem(df, bm, is_txt=False)
# ls_all_marsh = marsh_all_bm(df, bm, [])

In [73]:
df1 = df[df['id']==5]

In [77]:
df1['rs'].iloc[0]

'1ГП,1,1/15П'

In [115]:
ls = df['sm1'].tolist()

In [116]:
ls1 = []
for s1 in ls:
    if s1 is not np.nan:
        ls2 = s1.split(',')
        for s2 in ls2:
            s3 = s2[1:]
            if s3 not in ls1:
                ls1.append(s3)

In [117]:
write_marsh('c:/tt/str_name.txt', ls1)

In [37]:
for i in range(5):
    print(i)
    for j in range(100,105):
        print(j)
        if j==102:
            break
        # print(j)


0
100
101
102
1
100
101
102
2
100
101
102
3
100
101
102
4
100
101
102


In [ ]:
def marsh_sv_bm(dfm, bm):
    ls = marsh_all_bm(dfm, bm, [bm])
    ls7 = []
    for lm in ls:
        for m1 in lm:
            bm1 = dfm[dfm['id']==m1]['bm'].iloc[0]
            if bm1 not in ls7:
                ls7.append(bm1)
        m1 = lm[-1]
        em = dfm[dfm['id']==m1]['em'].iloc[0]
        if em not in ls7:
            ls7.append(em)
    write_marsh("marsh_sv_bm.txt", ls7)

def marsh_sv_bm1(dfm, ls_all_marsh):
    ls = []
    for lm in ls_all_marsh:
        for id_m in lm:
            bm1 = dfm[dfm['id']==id_m]['bm'].iloc[0]
            if bm1 not in ls:
                ls.append(bm1)
        id_m = lm[-1]
        em = dfm[dfm['id']==id_m]['em'].iloc[0]
        if em not in ls:
            ls.append(em)
    return ls

def marsh_rs_bm1(dfm, ls_all_marsh):
    ls = []
    for lm in ls_all_marsh:
        for id_m in lm:
            rs1 = dfm[dfm['id']==id_m]['rs'].iloc[0]
            if rs1 is not np.nan:
                ls_st = rs1.split(',')
                for rs in ls_st:
                    if rs not in ls:
                        ls.append(rs)
    return ls

def marsh_str_bm1(dfm, ls_all_marsh):
    ls = []
    for lm in ls_all_marsh:
        for id_m in lm:
            st1 = dfm[dfm['id']==id_m]['sm'].iloc[0]
            if st1 is not np.nan:
                ls_st = st1.split(',')
                for st2 in ls_st:
                    st3 = st2[1:]
                    if st3 not in ls:
                        ls.append(st3)
    return ls

In [8]:
ds = {
    0: {
        'rs': '3-5',
        'state': 0,
        'fl': 0
    },
    1: {
        'rs': '5/9П',
        'state': 0,
        'fl': 0
    },
    2: {
        'rs': '9',
        'state': 0,
        'fl': 0
    },
    3: {
        'rs': '11-19',
        'state': 0,
        'fl': 0
    },
    4: {
        'rs': '23',
        'state': 0,
        'fl': 0
    },
}

In [9]:
ds

{0: {'rs': '3-5', 'state': 0, 'fl': 0},
 1: {'rs': '5/9П', 'state': 0, 'fl': 0},
 2: {'rs': '9', 'state': 0, 'fl': 0},
 3: {'rs': '11-19', 'state': 0, 'fl': 0},
 4: {'rs': '23', 'state': 0, 'fl': 0}}

In [72]:
ds.get(4)

{'rs': '23', 'state': 0, 'fl': 1}

In [288]:
rs_n = []
def f0(num):
    err = 0
    global rs_n
    n1 = num-1
    for i in range(num, 5):
        st1 = ds[i]['state']
        fl1 = ds[i]['fl']
        if not (st1 == 0 and fl1 == 0):
            err = 1
            rs_n.append([[n1, ds[n1]['state'], ds[n1]['fl']],
                         [i, ds[i]['state'], ds[i]['fl']]])
            break
    return err

def f1(num):
    err = 0
    global rs_n
    n1 = num-1
    for i in range(num, 5):
        st1 = ds[i]['state']
        fl1 = ds[i]['fl']
        if not (st1 <= 1 and fl1 == 0):
            err = 1
            rs_n.append([[n1, ds[n1]['state'], ds[n1]['fl']],
                         [i, ds[i]['state'], ds[i]['fl']]])
            break
    return err

def f2(num):
    err = 0
    global rs_n
    n1 = num-1
    for i in range(num, 5):
        st1 = ds[i]['state']
        fl1 = ds[i]['fl']
        if not (st1 == 1 and fl1 == 1):
            err = 1
            rs_n.append([[n1, ds[n1]['state'], ds[n1]['fl']],
                         [i, ds[i]['state'], ds[i]['fl']]])
            break
    return err

# def f2(num):
#     global rs_n
#     fl2 = ds[num]['fl']
#     st2 = ds[num]['state']
#     n1 = num-1
#     if not (fl2 == 1 and st2 == 0):
#         rs_n.append([[n1, ds[n1]['state'], ds[n1]['fl']],
#                      [num, st2, fl2]])
#         return 1
#     return 0

def ff():
    err = 0
    global rs_n

    for i in range(5):
        st1 = ds[i]['state']
        fl1 = ds[i]['fl']

        if fl1 != 3:
            if i < 5-1:
                st2 = ds[i+1]['state']
                if fl1==0:
                    err = f0(i+1)
                    if st1==0 and st2==0:
                       pass
                    if st1==1 and st2==0:
                        ds[i]['fl'] = 1

                elif fl1==1:
                    err = f1(i+1)
                    if st1==1 and st2==1:
                        ds[i]['fl'] = 2

                elif fl1==2:
                    # err = f2(i+1)
                    if st1==0 and st2==1:
                        ds[i]['fl'] = 3
                    # err = f2(i+1)
                else:
                    err = 1
                    rs_n.append([[i, st1, fl1], [i+1, st2]])
            else:
                if fl1==0:
                    if st1==0:
                       pass
                    if st1==1:
                        ds[i]['fl'] = 1
                # elif fl1==1:
                #     if st1==1:
                #         ds[i]['fl'] = 2
                elif fl1==1:
                    if st1==0:
                        ds[i]['fl'] = 3
                else:
                    rs_n.append([[i, st1, fl1]])
                    err = 1

    fl = 1
    for i in range(5):
        if ds[i]['fl'] != 3:
            fl = 0

    if fl == 1:
        err = 2
    return err


In [126]:
import threading
from time import sleep

In [289]:
def f12():
    global  err
    while True:
        err = ff()
        if err > 0:
            break
        sleep(1)

def f13():
    global ls
    err = 0
    # ls = [[0,0], [0,1], [1,1], [0,0], [2,1], [1,0], [3,1], [2,0], [4,1], [3,0], [4,0]]
    for ls1 in ls:
        ds[ls1[0]]['state'] = ls1[1]
        sleep(3)

In [303]:
ds = {
    0: {
        'rs': '3-5',
        'state': 0,
        'fl': 0
    },
    1: {
        'rs': '5/9П',
        'state': 0,
        'fl': 0
    },
    2: {
        'rs': '9',
        'state': 0,
        'fl': 0
    },
    3: {
        'rs': '11-19',
        'state': 0,
        'fl': 0
    },
    4: {
        'rs': '23',
        'state': 0,
        'fl': 0
    },
}
err = 0
rs_n = []

ls = [[0,0], [0,1], [1,1],   [3,1],   [0,0], [1,0], [3,1], [2,0], [4,1], [3,0], [4,0]]

t = threading.Thread(target=f12)
t.start()
t1 = threading.Thread(target=f13)
t1.start()

In [314]:
ds

{0: {'rs': '3-5', 'state': 0, 'fl': 3},
 1: {'rs': '5/9П', 'state': 0, 'fl': 1},
 2: {'rs': '9', 'state': 0, 'fl': 0},
 3: {'rs': '11-19', 'state': 0, 'fl': 2},
 4: {'rs': '23', 'state': 0, 'fl': 1}}

False